# **Capstone Project** 
### ***The Battle of the Neighborhoods***

*Applied Data Science Capstone - IBM/Coursera*

#### Table of contents
- Introduction: Business Problem
- Data
- Methodology
- Analysis
- Results and Discussion
- Conclusion

### _Introduction: Business Problem_

Canada aims to invite 1 million immigrants over the next three years, the nation would need to welcome approximately 350,000 roughly 1 percent of its current population. Toronto is the provincial capital of Ontario and the most populous city in Canada, with a population of 2,731,571 as of 2016. The city continues to grow and attract immigrants. A study by Ryerson University showed that Toronto was the fastest-growing city in North America.

This project aims to aid immigrants to identify the right neighborhood to settle in Toronto, based on their personal preferences. The project will explore different localities of Toronto and identify various venues of interest in each neighborhood. 

Immigrating to a new country is a challenging task whether alone or with a family. Hence, choosing where to live is crucial for any immigrant and requires informed decision making.  This project will focus to resolve this challenge.

### _Data_
- List of postal codes in Canada through Wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

- Geospatial data of Toronto Neighborhoods in csv format (http://cocl.us/Geospatial_data)

- Foursquare Database for location information (http://foursquare.com)

Libraries

In [1]:
pip install bs4

     |████████████████████████████████| 112kB 8.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

     |████████████████████████████████| 102kB 14.0MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import requests
import json 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import folium 
from geopy.geocoders import Nominatim
print('Importation Complete.')

Importation Complete.


**Importing Data**

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

soup = BeautifulSoup(requests.get(url).text, 'html.parser')
rows = soup.body.table.tbody.text.split('\n\n\n')
data = []
for row in rows :
    cell = row.split('\n')
    if len(cell) == 3 :
        data.append(cell)
print('Number of neighbourhoods scraped: ', len(data))        
data[:5]

Number of neighbourhoods scraped:  286


[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront']]

**Data Wrangling**

In [5]:
df = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighbourhood'])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#Remove unassigned data cells
df = df[df['Borough'] != "Not assigned"].reset_index()
df.drop('index', axis = 1, inplace = True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [7]:
# Merge simmilar boroughs with unique postcodes
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()
print(df.shape)

(103, 3)


In [8]:
#merge lattitudes and longitudes from the csv file
G_data = pd.read_csv('http://cocl.us/Geospatial_data')
G_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df = pd.merge(df, G_data, on='PostalCode')
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Visualization of the Dataset in Map view**

In [9]:
# Find coordinates of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# Displaying location on the map
Toronto_Map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_Map)  
    
Toronto_Map

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Obtaining Information from Foursquare API**

In [10]:
# Defining Creditentials
CLIENT_ID = 'SQARXQRVULTRTJESUOPUSLUZMQO5Y5SRIQOGQUHEIKQ0CGHB'
CLIENT_SECRET = 'UAQO5PMEJYI2VAM3HKWQDU1YAZHI4TYZXWWLTE55I5AHLXRD' 
VERSION = '20180605'

In [11]:
# defining Venues within 500m radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
df_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

df_venues.head()
print(df_venues.shape)

(2241, 7)


In [13]:
# Venues per neighbourhood
df_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [14]:
print('There are {} unique categories.'.format(len(df_venues['Venue Category'].unique())))

There are 277 unique categories.


In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] =df_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2241, 278)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malv

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(100, 278)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, K

In [17]:
# Identifying top Venues for each neighbourhood
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.05
2           Bar  0.04
3    Steakhouse  0.04
4  Burger Joint  0.03


----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0         Playground  0.33
1               Park  0.33
2        Coffee Shop  0.33
3  Accessories Store  0.00
4      Metro Station  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.25
1          Pizza Place  0.12
2  Fried Chicken Joint  0.12
3             Pharmacy  0.12
4       Sandwich Place  0.12


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place

In [18]:

# function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Sushi Restaurant,Thai Restaurant,Salad Place,Restaurant,Burger Joint,Bakery
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Beer Store,Pharmacy,Fried Chicken Joint,Event Space,Ethiopian Restaurant,Empanada Restaurant
4,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Skating Rink,Pharmacy,Coffee Shop,Sandwich Place,Pub,Athletics & Sports,Doner Restaurant


**Clustering Neighbourhoods** 

In [20]:

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

In [21]:
kclusters = 5

# k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [22]:

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,History Museum,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Medical Center,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Intersection,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bakery,Donut Shop,Discount Store


In [23]:
toronto_merged = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])].reset_index(drop=True)

In [24]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,History Museum,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Medical Center,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Intersection,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bakery,Donut Shop,Discount Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Discount Store,Convenience Store,Bus Station,Coffee Shop,Department Store,Chinese Restaurant,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bus Line,Bakery,Soccer Field,Park,Intersection,Metro Station,Bus Station,Fast Food Restaurant,Donut Shop,Drugstore
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Motel,American Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,College Stadium,Skating Rink,General Entertainment,Café,Event Space,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner


**Visualization**

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
closest, _ = metrics.pairwise_distances_argmin_min(kmeans.cluster_centers_, toronto_grouped_clustering)
closest

array([ 0, 76, 74, 75, 26])

In [27]:
# function to get venues by their frequency in descending order
def get_venues_by_freq(val, repindex) :
    num_top_venues = val
    print(toronto_grouped.iloc[repindex, 0].upper()+': ' )
    temp = toronto_grouped.iloc[repindex, :].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_Cluster 1_

In [28]:
cluster1_df  = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
cluster1_df

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",0,History Museum,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
2,"Guildwood, Morningside, West Hill",0,Medical Center,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Intersection,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
3,Woburn,0,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Cedarbrae,0,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Athletics & Sports,Thai Restaurant,Fried Chicken Joint,Bakery,Donut Shop,Discount Store
5,Scarborough Village,0,Playground,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Convenience Store,Bus Station,Coffee Shop,Department Store,Chinese Restaurant,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
7,"Clairlea, Golden Mile, Oakridge",0,Bus Line,Bakery,Soccer Field,Park,Intersection,Metro Station,Bus Station,Fast Food Restaurant,Donut Shop,Drugstore
8,"Cliffcrest, Cliffside, Scarborough Village West",0,Motel,American Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant
9,"Birch Cliff, Cliffside West",0,College Stadium,Skating Rink,General Entertainment,Café,Event Space,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner
10,"Dorset Park, Scarborough Town Centre, Wexford ...",0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Brewery,Pet Store,Light Rail Station,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [29]:

get_venues_by_freq(20, closest[0])

ADELAIDE, KING, RICHMOND: 
                            venue  freq
0                     Coffee Shop  0.08
1                            Café  0.05
2                             Bar  0.04
3                      Steakhouse  0.04
4                    Burger Joint  0.03
5                          Bakery  0.03
6                Sushi Restaurant  0.03
7                 Thai Restaurant  0.03
8                     Salad Place  0.03
9                      Restaurant  0.03
10               Asian Restaurant  0.03
11                      Gastropub  0.02
12                   Concert Hall  0.02
13                    Pizza Place  0.02
14                 Cosmetics Shop  0.02
15  Vegetarian / Vegan Restaurant  0.02
16            American Restaurant  0.02
17                          Hotel  0.02
18                            Gym  0.02
19                          Plaza  0.01




_Cluster 2_

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [31]:
get_venues_by_freq(20, closest[1])

ROUGE, MALVERN: 
                              venue  freq
0              Fast Food Restaurant   1.0
1                 Accessories Store   0.0
2                     Metro Station   0.0
3   Molecular Gastronomy Restaurant   0.0
4        Modern European Restaurant   0.0
5                 Mobile Phone Shop   0.0
6                Miscellaneous Shop   0.0
7         Middle Eastern Restaurant   0.0
8                Mexican Restaurant   0.0
9                       Men's Store   0.0
10                    Luggage Store   0.0
11         Mediterranean Restaurant   0.0
12                   Medical Center   0.0
13                   Massage Studio   0.0
14                Martial Arts Dojo   0.0
15                           Market   0.0
16              Monument / Landmark   0.0
17                            Motel   0.0
18                    Movie Theater   0.0
19                           Museum   0.0




_Cluster 3_

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North, L'Amoreaux East, Milliken, St...",2,Park,Playground,Coffee Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
19,"Silver Hills, York Mills",2,Park,Cafeteria,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
23,Parkwoods,2,Park,Food & Drink Shop,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
28,"CFB Toronto, Downsview East",2,Park,Airport,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
38,East Toronto,2,Park,Coffee Shop,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
48,Rosedale,2,Park,Trail,Playground,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
72,Caledonia-Fairbanks,2,Park,Fast Food Restaurant,Women's Store,Market,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
88,"The Kingsway, Montgomery Road, Old Mill North",2,Park,Smoke Shop,River,Comfort Food Restaurant,Comic Shop,Falafel Restaurant,Event Space,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant
95,Weston,2,Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant


In [33]:
get_venues_by_freq(20, closest[2])

ROSEDALE: 
                              venue  freq
0                              Park  0.50
1                             Trail  0.25
2                        Playground  0.25
3                 Accessories Store  0.00
4   Molecular Gastronomy Restaurant  0.00
5        Modern European Restaurant  0.00
6                 Mobile Phone Shop  0.00
7                Miscellaneous Shop  0.00
8         Middle Eastern Restaurant  0.00
9                Mexican Restaurant  0.00
10                      Men's Store  0.00
11                    Metro Station  0.00
12                            Motel  0.00
13         Mediterranean Restaurant  0.00
14                   Medical Center  0.00
15                   Massage Studio  0.00
16                Martial Arts Dojo  0.00
17                           Market  0.00
18              Monument / Landmark  0.00
19                           Museum  0.00




_Cluster 4_

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Roselawn,3,Garden,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [35]:
get_venues_by_freq(20, closest[3])

ROSELAWN: 
                              venue  freq
0                            Garden   1.0
1                 Accessories Store   0.0
2                     Metro Station   0.0
3        Modern European Restaurant   0.0
4                 Mobile Phone Shop   0.0
5                Miscellaneous Shop   0.0
6         Middle Eastern Restaurant   0.0
7                Mexican Restaurant   0.0
8                       Men's Store   0.0
9               Monument / Landmark   0.0
10         Mediterranean Restaurant   0.0
11                   Medical Center   0.0
12                   Massage Studio   0.0
13                Martial Arts Dojo   0.0
14                           Market   0.0
15                 Malay Restaurant   0.0
16  Molecular Gastronomy Restaurant   0.0
17                            Motel   0.0
18                           Lounge   0.0
19                      Opera House   0.0




_Cluster 5_

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,"Cloverdale, Islington, Martin Grove, Princess ...",4,Golf Course,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [37]:

get_venues_by_freq(20, closest[4])

CLOVERDALE, ISLINGTON, MARTIN GROVE, PRINCESS GARDENS, WEST DEANE PARK: 
                              venue  freq
0                       Golf Course   1.0
1                 Accessories Store   0.0
2                             Motel   0.0
3   Molecular Gastronomy Restaurant   0.0
4        Modern European Restaurant   0.0
5                 Mobile Phone Shop   0.0
6                Miscellaneous Shop   0.0
7         Middle Eastern Restaurant   0.0
8                Mexican Restaurant   0.0
9                     Metro Station   0.0
10                      Men's Store   0.0
11         Mediterranean Restaurant   0.0
12                   Medical Center   0.0
13                   Massage Studio   0.0
14                Martial Arts Dojo   0.0
15                           Market   0.0
16                 Malay Restaurant   0.0
17              Monument / Landmark   0.0
18                    Movie Theater   0.0
19                           Lounge   0.0




In [38]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Discussion
Each cluster developed in this project was able to successfully distinguish itself with unique venues. Most populated cluster 1, identifies itself as the choice for foodies. Anyone who enjoys experiencing different cuisines or favors close proximity to restaurants would prefer this cluster. Commuters or office workers who enjoy a cup of coffee on the morning, would appreciate living in these neighbourhoods as the most common venue are coffee shops.

The 2nd biggest cluster of the study, cluster 3, has parks as the number 1 most common venue in all its neighbourhoods. Along with trails and playgrounds, this cluster will be preferred by people who seek nature, walks and outdoor activities. Moreover, families will also gravitate towards this cluster for having access to playgrounds for their kids.  

The other 3 clusters are outliers with unique venues. Cluster 5 neighbourhoods are represented by golf courses, which will be appreciated by golf enthusiasts. Whereas, Cluster 4 with the single neighbourhood namely Roselawn is a cemetery, which tends to be relatively cheaper to live nearby. Lastly, Cluster 2 has fast food restaurant as the most common venue along with yoga studio, dog run and electronic/accessory store, which seems like the most diverse set of amenities for anyone looking for convenience.


## Conclusion
This study aims to aid immigrants and new commers to Toronto city, which is the most rapidly growing metropolitan city of North America. This will help to have a better understanding of the residential areas with respect to the most common venues in that neighbourhood. In this age, having more information and options is vital to making an informed decision. 

The project has highlighted 5 clusters with unique attributes that may help decide an individual or a family to relocate based on their personal preference. For instance, if a person is looking for a neighborhood with diverse and easily accessible dining options then Cluster 1 would be recommended. For a family or outdoors loving person, the neighborhoods in Cluster 3 are more suitable, due to the common venues such as parks, playgorunds and trails which are ideal for kids and people who love outdoor activities. For people who are looking for something specific like golf, then cluster 5 is the best fit for their lifestyle. 

The selection of neighborhoods may differ from person to person based on their needs. This study can be further analyzed and refined by including other factors such as crime rate, cost and school districts
